In [ ]:
import tensorflow as tf
from Dive_into_deep_learning.d2l import tensorflow as d2l

In [ ]:
class AttentionDecoder(d2l.Decoder):
    """The base attention-based decoder interface"""
    def __init__(self):
        super().__init__()
        
    @property
    def attention_weights(self):
        raise NotImplementedError

In [ ]:
class Seq2SeqAttentionDecoder(AttentionDecoder):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, dropout=0):
        super().__init__()
        self.attention = d2l.AdditiveAttention(num_hiddens, num_hiddens, num_hiddens, dropout)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embed_size)
        self.rnn = tf.keras.layers.RNN(tf.keras.layers.StackedRNNCells(
            [tf.keras.layers.GRUCell(num_hiddens, dropout=dropout) for _ in range(num_layers)]
        ), return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
        
    def init_state(self, enc_outputs, enc_valid_lens):
        outputs, hidden_state = enc_outputs
        return tf.transpose(outputs, (1, 0, 2)), hidden_state, enc_valid_lens
    
    def call(self, X, state, **kwargs):
        enc_outputs, hidden_state, enc_valid_lens = state